In [30]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [31]:
df = pd.read_csv("train_dataset_train.csv")
test = pd.read_csv('test_dataset_test.csv')

/tmp/ipykernel_117650/398589533.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_dataset_train.csv")
/tmp/ipykernel_117650/398589533.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test_dataset_test.csv')


In [34]:
df["is_in_yandex"] = pd.Categorical(df["is_in_yandex"])
df["is_in_yandex"].astype('category').cat.codes
df["is_in_yandex"] = df["is_in_yandex"].cat.codes

In [35]:
test["is_in_yandex"] = pd.Categorical(test["is_in_yandex"])
test["is_in_yandex"].astype('category').cat.codes
test["is_in_yandex"] = test["is_in_yandex"].cat.codes

In [36]:
df["is_return"] = pd.Categorical(df["is_return"])
df["is_return"].astype('category').cat.codes
df["is_return"] = df["is_return"].cat.codes

In [37]:
test["is_return"] = pd.Categorical(test["is_return"])
test["is_return"].astype('category').cat.codes
test["is_return"] = test["is_return"].cat.codes

Обьединим список не нужных строк с списком строк типа object

In [42]:
col_obj = df.select_dtypes(include=['object']).columns.values
col_obj = list(set(col_obj) ^ set(["id", "label"]))

In [43]:
col_obj

['type',
 'index_oper',
 'label',
 'is_privatecategory',
 'name_mfi',
 'oper_type + oper_attr',
 'id']

In [44]:
X = df.drop(col_obj, axis = 1)
y = df[["label"]]

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Обучение модели

In [46]:
import lightgbm as lgb

In [148]:
clf = lgb.LGBMClassifier(num_leaves=1023, random_state=42)

In [149]:
clf.fit(X_train, y_train)

/home/dro/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dro/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(num_leaves=1023, random_state=42)

## Оценка точности

In [150]:
pred = clf.predict_proba(X_test)[:, 1] > 0.01

In [151]:
from sklearn.metrics import recall_score

score = recall_score(y_test, pred, average = "macro" )

In [152]:
print("Recall", score)

Recall 0.8720089566982863


In [153]:
ss = pd.read_csv('sample_solution.csv')

In [154]:
ss['label'] = (clf.predict_proba(test[X_test.columns])[:, 1] > 0.01).astype(int)

In [155]:
ss.to_csv('solut.csv', index=False)

In [156]:
ss['label'].mean()

0.251033